In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from patsy import dmatrices
import statsmodels.api as sm

from src.evb import get_evb_incidence
from src.rki import get_consultations

In [ ]:
evb = get_evb_incidence().set_index('Kalenderwoche').rename(columns={ 'EVB': 'incidence' })
evb['group_is_treatment'] = 1
consultations = get_consultations().set_index('Kalenderwoche').rename(columns={ 'Konsultationen': 'incidence' })
consultations['group_is_treatment'] = 0

In [ ]:
def model_did(start, end, intervention):
    df = pd.concat([
        evb.loc[pd.to_datetime(start):pd.to_datetime(end)],
        consultations.loc[pd.to_datetime(start):pd.to_datetime(end)]
    ]).reset_index()
    df['after_treatment'] = (df['Kalenderwoche'] > pd.to_datetime(intervention)).astype(int)
    y_train, X_train = dmatrices('incidence ~ is_treatment + after_treatment + is_treatment * after_treatment', df, return_type='dataframe')
    did_model = sm.OLS(endog=y_train, exog=X_train)
    did = did_model.fit()
    
    df['prediction'] = did.predict()

    grouped = df.set_index('Kalenderwoche').groupby('is_treatment')
    colors = get_standard_colors(num_colors=len(grouped))
    fig, ax = plt.subplots(figsize=(10, 5))
    axs = [ax, ax.twinx()]

    for (label, df), color, ax in zip(grouped, colors, axs):
        name = 'EVB' if label == 1 else 'Konsultationen'
        df['incidence'].plot(ax=ax, label=name, color=color)
        df['prediction'].plot(ax=ax, label='mean', color=color, alpha=0.6)

    for ax, label in zip(axs, ['Konsultationen', 'EVB']):
        ax.set_ylabel(label)
        ax.legend()

    return did

In [ ]:
model_did('2021-06', '2021-08-31', '2021-07-21').summary()